In [1]:
import numpy as np 

In [2]:
class Layer:
    def __init__(self, n_input, n_neurons, activation = None, weights = None, bias = None):
        """
        :param int n_input: input nodes
        :param int n_neurons: output nodes
        :param str activation: activation function
        :param weights: weight vectors
        :param bias: bias vectors
        """
        self.weights = weights if weights is not None else np.random.rand(n_input, n_neurons)* np.sqrt(1/n_neurons)
        self.bias = bias if bias is not None else np.random.rand(n_neurons)
        self.activation = activation
        self.last_activation = None
        self.error = None
        self.delta = None 
    
    def activate(self, x):
        r = np.dot(x, self.weights) + self.bias
        self.last_activation = self._apply_activation(r)
        return self.last_activation

    def _apply_activation(self, r):
        if self.activation == None:
            return r #no activation function
        elif self.activation == 'relu':
            return np.maximum(r,0)
        elif self.activation == 'tanh':
            return np.tanh(r)
        elif self.activation == 'sigmoid':
            return 1 / (1 + np.exp(-r))
        return r
    def apply_activation_derivatives(self, r):
        if self.activation is None:
            return np.ones_like(r)
        elif self.activation == 'Relu':
            grad = np.array(r, copy=True)
            grad[r > 0] = 1.
            grad[r <= 0] = 0.
            return grad
        elif self.activation == 'tanh':
            return 1 - r ** 2
        elif self.activation == 'sigmoid':
            return r*(1-r)
        return r

In [11]:
class NeuralNetwork:
    # Neural Network Class
    def __init__(self):
        self._layers = []  # list of network class
    def add_layer(self, layer):
        # Add layers
        self._layers.append(layer)
    def feed_forward(self, X):
        # Forward calculation
        for layer in self._layers:
            # Loop through every layer
            X = layer.activate(X)
        return X
    def backpropagation(self, X, y, learning_rate):
          # Back propagation
        # Get result of forward calculation
        output = self.feed_forward(X)
        for i in reversed(range(len(self._layers))):
            layer = self._layers[i]
            # If it’s output layer
            if layer == self._layers[-1]:
                layer.error = y - output
                # calculate delta
                layer.delta = layer.error * layer.apply_activation_derivative(output)
            else:
                next_layer = self._layers[i + 1]
                layer.error = np.dot(next_layer.weights,next_layer.delta)
                 # Calculate delta
                layer.delta = layer.error * layer.apply_activation_derivative(layer.last_activation)
        for i in range(len(self._layers)):
            layer = self._layers[i]
                      # o_i is output of previous layer
            o_i = np.atleast_2d(X if i == 0 else self._layers[i - 1].last_activation)
                      # Gradient descent
            layer.weights += layer.delta * o_i.T * learning_rate
    def train(self, X_train, X_test, y_train, y_test, learning_rate, max_epochs):
        # Train network
        # one-hot encoding
        y_onehot = np.zeros((y_train.shape[0], 2))
        y_onehot[np.arange(y_train.shape[0]), y_train] = 1
        mses = []
        for i in range(max_epochs):
            for j in range(len(X_train)):
                self.backpropagation(X_train[j], y_onehot[j],learning_rate)
            if i % 10 == 0:
                mse = np.mean(np.square(y_onehot - self.feed_forward(X_train)))
            mses.append(mse)
            print('Epoch: #%s, MSE: %f' % (i, float(mse)))
            print('Accuracy: %.2f%%' % (self.accuracy(self.predict(X_test), y_test.flatten()) * 100))
        return mses

In [5]:
nn = NeuralNetwork()
nn.add_layer(Layer(2, 25, 'sigmoid'))  # Hidden layer 1, 2=>25
nn.add_layer(Layer(25, 50, 'sigmoid')) # Hidden layer 2, 25=>50
nn.add_layer(Layer(50, 25, 'sigmoid')) # Hidden layer 3, 50=>25
nn.add_layer(Layer(25, 2, 'sigmoid'))  # Hidden layer, 25=>2